In [35]:
%pip install -r requirements.txt

  Using cached langdetect-1.0.9-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# scrape https://www.poetryfoundation.org/poems/browse#page=1&sort_by=recently_added for 40,000 poems?

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import logging
import re

In [41]:
df = pd.read_pickle('poets_org.pickle')
print(len(df))

13877


In [46]:
for i, poem in enumerate(df.content):
    if poem == '':
        df.drop(index=i, inplace=True)
df.reset_index(drop=True, inplace=True)

In [47]:
len(df)

13823

In [48]:
df.to_pickle('poets_org.pickle')

In [49]:
df1 = pd.read_pickle('mostly_english_poems.pickle')
print(len(df1))

16464


In [50]:
df = pd.read_pickle('filtered_poems.pickle')
print(len(df))

13674


In [51]:
frames = [df, df1]

In [52]:
df = pd.concat(frames)

In [53]:
df.reset_index(drop=True, inplace=True)

In [54]:
df.content

0        Do not go gentle into that good night,\nOld ag...
1        How do I love thee? Let me count the ways.\nI ...
2        Shall I compare thee to a summer’s day?\nThou ...
3        If you can keep your head when all about you\n...
4        Nature’s first green is gold,\nHer hardest hue...
                               ...                        
30133    The happiest day--the happiest hour\n  My sear...
30134    Wreathed in myrtle, my sword I'll conceal,\n  ...
30135    Oh! that my young life were a lasting dream!\n...
30136    _How often we forget all time, when lone\nAdmi...
30137    How shall the burial rite be read?\n  The sole...
Name: content, Length: 30138, dtype: object

In [69]:
df = pd.read_pickle('filtered_poems.pickle')

In [70]:
df.reset_index(drop=True, inplace=True)

In [71]:
seen_title = set()
seen_auth = set()
for i in range(len(df)):
    title = df.title[i]
    author = df.author[i]
    if title in seen_title and author in seen_auth:
        df.drop(index=i, inplace=True)
    else:
        seen_title.add(title)
        seen_auth.add(author)

In [72]:
df.reset_index(drop=True, inplace=True)
print(len(df))

26817


In [73]:
df.to_pickle('mostly_english_poems.pickle')

In [74]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oliver\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [75]:
Poems = df.content.map(lambda x: nltk.tokenize.word_tokenize(x))

In [76]:
Poems[1]

['How',
 'do',
 'I',
 'love',
 'thee',
 '?',
 'Let',
 'me',
 'count',
 'the',
 'ways',
 '.',
 'I',
 'love',
 'thee',
 'to',
 'the',
 'depth',
 'and',
 'breadth',
 'and',
 'height',
 'My',
 'soul',
 'can',
 'reach',
 ',',
 'when',
 'feeling',
 'out',
 'of',
 'sight',
 'For',
 'the',
 'ends',
 'of',
 'being',
 'and',
 'ideal',
 'grace',
 '.',
 'I',
 'love',
 'thee',
 'to',
 'the',
 'level',
 'of',
 'every',
 'day',
 '’',
 's',
 'Most',
 'quiet',
 'need',
 ',',
 'by',
 'sun',
 'and',
 'candle-light',
 '.',
 'I',
 'love',
 'thee',
 'freely',
 ',',
 'as',
 'men',
 'strive',
 'for',
 'right',
 '.',
 'I',
 'love',
 'thee',
 'purely',
 ',',
 'as',
 'they',
 'turn',
 'from',
 'praise',
 '.',
 'I',
 'love',
 'thee',
 'with',
 'the',
 'passion',
 'put',
 'to',
 'use',
 'In',
 'my',
 'old',
 'griefs',
 ',',
 'and',
 'with',
 'my',
 'childhood',
 '’',
 's',
 'faith',
 '.',
 'I',
 'love',
 'thee',
 'with',
 'a',
 'love',
 'I',
 'seemed',
 'to',
 'lose',
 'With',
 'my',
 'lost',
 'saints',
 '.',
 'I'

In [77]:
def BiGram(dic, poem):
    a = poem
    b = poem[1:]
    for i in range(len(b)):
        if(a[i] in dic):
            if(b[i] in dic[a[i]]):
                dic[a[i]][b[i]] = dic[a[i]][b[i]]+1
            else:
                dic[a[i]][b[i]] = 1
        else:
            dic[a[i]]={b[i]:1}
            
    return(dic)

In [78]:
dicBi = {} # The dictionary for the BiGram model
for poem in Poems: #Feed it all the poems
    dicBi = BiGram(dicBi, poem) #Build the model!

In [79]:
def Next_word(word, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)

In [80]:
prev_word = "how"
generate_str = prev_word
for i in range(20):
    next_word = Next_word(prev_word, dicBi)
    prev_word = next_word
    generate_str = generate_str+" "+ next_word

In [81]:
generate_str

'how to the world , and the world , and the world , and the world , and the world ,'

In [82]:
def NGram(dictionary, poem, nGram):
    Number_of_Ngrams = len(poem)-nGram+1

    for position in range(Number_of_Ngrams):
        words = [] 
        for nWord in range(nGram):
            words = words + [poem[nWord+position]]
        temp_dic = dictionary

        for nWord in range(nGram):
            current_word = words[nWord]
            last_word = nWord+1==nGram
            if(current_word in temp_dic):
                if(last_word):
                    temp_dic[current_word] = temp_dic[current_word]+1 #Increase the Ngram Count by 1
                else:    
                    temp_dic = temp_dic[current_word]
            else:
                create_dic = 0
                if(last_word):
                    create_dic = 1
                else:
                    create_dic = {words[-1]:1}
                
                for k in range(nGram-2,nWord,-1):
                    create_dic = {words[k]:create_dic}
                temp_dic[current_word] = create_dic
                break
    return(dictionary)

In [83]:
dic4 = {}
for i in Poems:
    dic4 = NGram(dic4, i, 4)

In [88]:
def Next_word3(word1,word2,word3, dic):
    Max = 0
    next_word = ""
    for i,j in list(dic[word1][word2][word3].items()):
        if(j>Max):
            Max = j
            next_word = i
    return(next_word)
#Next_word3("NEWLINE","love","is",dic4)

In [90]:
prev_word1 = "what"
prev_word2 = "is"#"Next_word(prev_word1, dic)"
prev_word3 = "this"# Next_word2(prev_word1,prev_word2, dic2)
generate_str = prev_word1+" "+prev_word2 + " " +prev_word3
for i in range(50):
    next_word = Next_word3(prev_word1, prev_word2,prev_word3, dic4)
    prev_word1 = prev_word2
    prev_word2 = prev_word3
    prev_word3 = next_word
    generate_str = generate_str+" "+ next_word
print(generate_str.replace("NEWLINE ","\n"))

what is this ? What nerve agent did Rembrandt hide within the dark shades of paint that he used ? What inside me had malfunctioned , had left me kneeling and sobbing in a museum , slowly rots away and its parts are gradually replaced . Eventually none of the words we say


In [86]:
# how should i go about generating new lines

In [87]:
# log the devices being used for tensorflow operations
tf.debugging.set_log_device_placement(True)